In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pickle
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [0]:
from matplotlib import pyplot as plt

In [0]:
cnn_checkpoint_path = "/content/drive/My Drive/Colab Notebooks/CS7IS2 - Artificial Intelligence/cnn_model.h5"
features_path = "/content/drive/My Drive/Colab Notebooks/CS7IS2 - Artificial Intelligence/features"
labels_path = "/content/drive/My Drive/Colab Notebooks/CS7IS2 - Artificial Intelligence/labels"
logs_directory_path = "/content/drive/My Drive/Colab Notebooks/CS7IS2 - Artificial Intelligence/logs"
NumberOfClasses = 15

In [0]:
def cnn_model(x, y, classes):
  # classes = 15
  model = Sequential()
  model.add(Conv2D(32, (5, 5), input_shape=(x,y,1), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
  model.add(Conv2D(64, (5, 5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.6))
  model.add(Dense(classes, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # file_path = "/content/drive/My Drive/Colab Notebooks/CS7IS2 - Artificial Intelligence/Qickdraw.h5"
  checkpoint = ModelCheckpoint(cnn_checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
  callbacks_list = [checkpoint]

  return model, callbacks_list

In [0]:
def load_data():
  file = open(features_path,"rb")
  try:
    features = np.array(pickle.load(file))
  finally:
    file.close()

  file1 = open(labels_path, "rb")
  try:
    labels = np.array(pickle.load(file1))
  finally:
    file1.close()

  return features, labels

In [0]:
def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels

In [0]:
def preprocessLabels(labels):
  labels = np_utils.to_categorical(labels)
  return labels

In [63]:
features, labels = load_data()
features, labels = sk.utils.shuffle(features, labels)
print ("features.shape: ", features.shape)
print ("labels.shape: ", labels.shape)

features.shape:  (150000, 784)
labels.shape:  (150000, 1)


In [64]:
labels = preprocessLabels(labels)
print ("labels.shape: ", labels.shape)

labels.shape:  (150000, 15)


In [0]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0, test_size=0.1)

In [66]:
print (train_x.shape)
print (test_x.shape)
print (train_y.shape)
print (test_y.shape)

(135000, 784)
(15000, 784)
(135000, 15)
(15000, 15)


In [67]:
train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
print ("test_x.shape: ", test_x.shape)
print ("train_x.shape: ", train_x.shape)

test_x.shape:  (15000, 28, 28, 1)
train_x.shape:  (135000, 28, 28, 1)


In [68]:
modelCNN, callbacks_list = cnn_model(28, 28, NumberOfClasses)
print_summary(modelCNN)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [69]:
 modelCNN.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=64, callbacks=[TensorBoard(log_dir=logs_directory_path)])

Train on 135000 samples, validate on 15000 samples
Epoch 1/3
135000/135000 [==============================] - 20s 147us/step - loss: 0.4749 - acc: 0.8638 - val_loss: 0.2639 - val_acc: 0.9257
Epoch 2/3
135000/135000 [==============================] - 19s 142us/step - loss: 0.2716 - acc: 0.9237 - val_loss: 0.2128 - val_acc: 0.9403
Epoch 3/3
135000/135000 [==============================] - 19s 142us/step - loss: 0.2275 - acc: 0.9359 - val_loss: 0.2108 - val_acc: 0.9397


In [0]:
modelCNN.save(cnn_checkpoint_path)

In [0]:
cnn_score = modelCNN.evaluate(test_x, test_y, verbose=0)

In [0]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0, test_size=0.1)

In [0]:
# Implementing K-Means
kmeans = KMeans(n_clusters = 5, random_state=0)

In [75]:
kmeans.fit(train_x, train_y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [0]:
kmeans_score = kmeans.score(test_x, test_y)

In [77]:
# Random Forrest
classifier = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
classifier.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
rfc_score = classifier.score(test_x, test_y)

In [0]:
# # Multi-Layer Perceptron (MLP)

# classifier = MLPClassifier(hidden_layer_sizes=784, random_state=0)

In [0]:
# classifier.fit(train_x, train_y)

In [0]:
# mlp_score = classifier.score(test_x, test_y)

In [82]:
print ("CNN Loss: ", cnn_score[0])
print ("CNN Accuracy: ", cnn_score[1])
print ("K-Means score: " , kmeans_score)
print ("Random Forest Classifier Score: ", rfc_score)
# print ("Multi Layer Perceptron Classifier Score: ", mlp_score)

CNN Loss:  0.21080010418494544
CNN Accuracy:  0.9397333333015442
K-Means score:  -837605.75
Random Forest Classifier Score:  0.7010666666666666
